# Generate *.npy Data with data_utils.py

Generate data.pkl, data_age.pkl, and data_gender.pkl. （see pickle python obj serialization)


In [ ]:
import numpy as np
import cv2
import os
import pandas as pd
from six.moves import cPickle
#file op
import shutil

In [ ]:
import sys
from google.colab import drive
drive.mount('/content/drive')
sys.path += '/content/drive/MyDrive/csb185_boneAge'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Check GPU, RAM condition
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Thu Mar 10 13:20:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pwd
root_dir = '/content/drive/MyDrive/csb185_boneAge/data_rsna_kaggle/'
train_dir = os.path.join(root_dir,'boneage-training-dataset/boneage-training-dataset/')
checkpointt_dir = os.path.join(mydrive,'weights2/')

/content


# generate initial data files


Skip this part by loading the generated files 

In [ ]:
#For this problem the validation and test data provided by the concerned authority did not have labels
#so the training data was split into train, test and validation sets data directory
# Original RSNA-Kaggle dataset contains = 12611 images

X_train = []
y_age = []
y_gender = []

df = pd.read_csv(os.path.join(root_dir,'boneage-training-dataset.csv')) 
a = df.to_numpy()
m = a.shape[0]


In [ ]:
path = train_dir  #split from above read.csv() for better running
print ('Loading data set...')

##= with a smaller sample, with random 50% of 15GB data (below 3 lines)
files = [f for f in os.listdir(path)]
random_files = np.random.choice(files, int(len(files)/12611*1390))


Loading data set...


In [ ]:
file_pkl = open(os.path.join(root_dir,'file.pkl'),'wb')
cPickle.dump(random_files, file_pkl, protocol=cPickle.HIGHEST_PROTOCOL)
file_pkl.close()

In [ ]:
len(random_files)

1390

In [ ]:
for i in random_files:
#for i in os.listdir(path): 
    y_age.append(df.boneage[df.id == int(i[:-4])].tolist()[0])
    a = df.male[df.id == int(i[:-4])].tolist()[0]
    if a:
        y_gender.append(1)
    else:
        y_gender.append(0)
    img_path = path + i
    shutil.copy(img_path, root_dir+'temp_train/')   #save to local path
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(300,300))
    x = np.asarray(img, dtype=np.uint8)
    X_train.append(x)
    
print ('100% completed loading data')

100% completed loading data


In [ ]:
len(X_train)

1390

In [ ]:
# Save data to .pkl files
train_pkl = open(os.path.join(root_dir,'data.pkl'),'wb')
cPickle.dump(X_train, train_pkl, protocol=cPickle.HIGHEST_PROTOCOL)
train_pkl.close()

train_age_pkl = open(os.path.join(root_dir,'data_age.pkl'),'wb')
cPickle.dump(y_age, train_age_pkl, protocol=cPickle.HIGHEST_PROTOCOL)
train_age_pkl.close()

train_gender_pkl = open(os.path.join(root_dir,'data_gender.pkl'),'wb')
cPickle.dump(y_gender, train_gender_pkl, protocol=cPickle.HIGHEST_PROTOCOL)
train_gender_pkl.close()


In [ ]:
len(list(os.listdir(os.path.join(root_dir,'temp_train'))))

1321

#Run main_classification.py to train classification model

In [ ]:
!pip install visualization

In [ ]:
%cd /content/drive/MyDrive/csb185_boneAge
from func_utils import *

In [ ]:
import pickle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import keras
import tensorflow
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.layers import Flatten, Dense, Input, Reshape, Lambda
from keras import backend as K
from keras import optimizers

os.environ["CUDA_VISIBLE_DEVICES"]="2"
os.environ['OMP_NUM_THREADS']='6'
batch_size = 32
epochs = 30


In [ ]:
# Load data
f = open(root_dir+'data_age.pkl', 'rb')
y = pickle.load(f)
f.close()

f = open(root_dir+'data_gender.pkl','rb')
gender = pickle.load(f)
f.close()

print('...loading training data')
f = open(root_dir+'data.pkl', 'rb')
x = pickle.load(f)
f.close()

x = np.asarray(x, dtype=np.float32) #takes 18GB of RAM
y = np.asarray(y)
gender = np.asarray(gender)

x /= 255.
gender =2*( gender-0.5) #1, 0 -> 1, -1 M, F
x_final = []
y_final = []
gender_final = []

...loading training data


In [ ]:
print('sample size: ', x.shape, ' age size: ', y.shape, ' gender size: ', gender.shape)

In [ ]:
# Shuffle images and split into train, validation and test sets
#random_no = np.random.choice(x.shape[0], size=x.shape[0], replace=False)
random_no = np.arange(x.shape[0])
np.random.seed(0)
np.random.shuffle(random_no)
for i in random_no:
    x_final.append(x[i,:,:,:])
    y_final.append(y[i])
    gender_final.append(gender[i])

x_final = np.asarray(x_final)
y_final = np.asarray(y_final)
gender_final = np.asarray(gender_final)

k = 55 # Decides split count 
x_test = x_final[:k,:,:,:]
y_test = y_final[:k]
gender_test = gender_final[:k]

x_valid = x_final[k:2*k,:,:,:]
y_valid = y_final[k:2*k]
gender_valid = gender_final[k:2*k]

x_train = x_final[2*k:,:,:,:]
y_train = y_final[2*k:]
gender_train = gender_final[2*k:]

#y_test = keras.utils.to_categorical(y_test,240)
#y_train = keras.utils.to_categorical(y_train,240)
#y_valid = keras.utils.to_categorical(y_valid,240)
y_train = softlabel(y_train,240) #see func utils
y_valid = softlabel(y_valid,240)
y_test = softlabel(y_test,240)
print (y_train[0,:])

In [ ]:
print ('x_train shape:'+ str(x_train.shape))
print ('y_train shape:'+ str(y_train.shape))
print ('gender_train shape:'+ str(gender_train.shape))
print ('x_valid shape:'+ str(x_valid.shape))
print ('y_valid shape:'+ str(y_valid.shape))
print ('gender_valid shape:' + str(gender_valid.shape))
print ('x_test shape:'+ str(x_test.shape))
print ('y_test shape:'+ str(y_test.shape))

Concatenate gender


In [ ]:
# Using VGG19 with pretrained weights from Imagenet but stil gender is embedded and concact'd
base_model = InceptionV3(weights='imagenet', include_top=False)
for i,layer in enumerate(base_model.layers):
    print (i,layer.name)
input = Input(shape=(300,300,3),name='input1')
input_gender = Input(shape=(1,),dtype='float32',name='input2')
output = base_model(input)
gender_embedding=Dense(16)(input_gender)
#gender_embedding=Dense(12)(gender_embedding)
#x = keras.layers.MaxPooling2D(pool_size=(3,3))(output)
#x = keras.layers.Conv2D(512,kernel_size=(3,3))(x)
#x = keras.layers.Conv2D(256,kernel_size=(1,1))(x)
print (K.int_shape(output))
x = keras.layers.MaxPooling2D(pool_size=(8,8))(output)
print (K.int_shape(x))
x = Flatten()(x)
f = keras.layers.Concatenate(axis=1)([x,gender_embedding])
print (K.int_shape(f)) 
#x = Dense(256, activation='relu')(x)
predictions = Dense(240)(x)


compile model

In [ ]:
model = Model(inputs=[input,input_gender], outputs=predictions)
for i,layer in enumerate(model.layers):
    print(i,layer.name)

Adam = tensorflow.keras.optimizers.Adam(learning_rate=0.0003,beta_1=0.9,beta_2=0.999)
model.compile(optimizer=Adam, loss='mean_absolute_error', metrics=['MAE'])



In [ ]:

# Save weights after every epoch
checkpoint = keras.callbacks.ModelCheckpoint(filepath=root_dir+'weights/weights.{epoch:02d}-{val_loss:.2f}.hdf5',save_weights_only=True)
history=model.fit([x_train,gender_train],y_train,batch_size=batch_size,epochs=60,verbose=1,validation_data=([x_valid,gender_valid],y_valid), callbacks = [checkpoint])
score = model.evaluate([x_test,gender_test], y_test, batch_size=batch_size)
print('Test loss:', score[0])
print('Test MAE:', score[1])

#TestMAE = TestMAE(model,x_test,y_test,gender_test)
#print ('TestMAE:',TestMAE)
##Visulization
weights = model.layers[-1].get_weights()[0]
print(weights.shape)

In [ ]:

#for layer in base_model.layers[:16]:
#    layer.trainable=False
#for layer in base_model.layers:
#    print (layer.name,layer.trainable)
Adam = tensorflow.keras.optimizers.Adam(learning_rate=0.0001,beta_1=0.9, beta_2=0.999)
model.compile(optimizer=Adam, loss='mean_absolute_error', metrics=['MAE'])
history = model.fit([x_train,gender_train],y_train,batch_size=batch_size,epochs=30,verbose=1,validation_data=([x_valid,gender_valid],y_valid), callbacks = [checkpoint])
score = model.evaluate([x_test,gender_test], y_test, batch_size=batch_size)
print('Test loss:', score[0])
print('Test MAE:', score[1])

#TestMAE = TestMAE(model,x_test,y_test,gender_test)
#print ('TestMAE:',TestMAE)

weights = model.layers[-1].get_weights()[0]
print (weights.shape)


In [ ]:
model.save_weights(os.path.join(root_dir,"model.h5"))
with open(os.path.join(root_dir,'history.pkl'), 'wb') as f:
	pickle.dump(history.history, f)
f.close()



# Generate Heatmaps for hand region, Region-1

**GAPAttention(model,weights,image_path)**  
heatmap saved in image_path  
attentionimg = 0.5 x heatmap + img  
**Region 1**  
Train the classification model with the original images resized to 560 × 560. The activation outputs (18×18×2048) are then fed into a GMP layer followed by the last FC layer. Derived by applying a binary mask of threshold = 50 (empiracally determined.  
**Region 2:**  
Trained with original images - Region 1. 





In [ ]:
t_dir = os.path.join(root_dir,'temp_train/')

In [ ]:
len(list(os.listdir(t_dir)))

1321

In [ ]:
#weight reload
import h5py
wf = h5py.File(os.path.join(root_dir,"model.h5"), 'r')
print(list(wf.keys()))

['dense_1', 'flatten', 'inception_v3', 'input1', 'input2', 'max_pooling2d_4', 'top_level_model_weights']


In [ ]:
# someone overrides the file i think
model.load_weights(os.path.join(root_dir,"modelver.h5"))
weights = model.layers[-1].get_weights()[0]
print(weights.shape)#2048240

In [ ]:
#reload history for graph
f = open(os.path.join(root_dir,'history.pkl'), 'rb')
history = pickle.load(f)
f.close()


In [ ]:
layer=K.function(model.layers[0].input, [model.layers[1].get_output_at(-1), model.layers[-1].output])
layer

<function keras.backend.function.<locals>.func>

In [ ]:
#readd file based on x
y.shape

(1390,)

In [ ]:
#because GAP is used instead of GMP for hand localization
def GAPAttention(model,weights,image_path):
    file_list = os.listdir(image_path)
    file_list.sort()
    for filename in file_list:
        filepath=image_path+filename
        print(filepath)
        image=load_image(filepath)
        image = image/255.0
        gender=1.0
        gender=np.asarray(gender)
        gender=np.expand_dims(gender,axis=0)
        #backend.function(input, output, update=None): Taking input from the first parameter 
        #and extracting the number of outputs as per the layers mentioned in the second parameter
        layer=K.function([model.layers[0].input, model.layers[4].input], [model.layers[1].get_output_at(-1), model.layers[-1].output])
        print(model.layers[0].input.shape)
        #Model(inputs=[input,input_gender]) 
        GAP,prediction=layer([image,gender])
        GAP=np.squeeze(GAP,axis=0)
        print(GAP.shape)
        index = np.argmax(prediction)
        print(index)
        #weight = weights[:,index]
        weight =np.mean(weights[:,index-5:index+5],axis=1)
        heatmap = np.zeros((GAP.shape[0],GAP.shape[1]))
        for k in range(GAP.shape[2]):
            heatmap = heatmap + weight[k]*GAP[:,:,k]
        heatmap = heatmap/np.max(heatmap)
        heatmap = np.uint8(255*heatmap)
        #print(heatmap.shape)
        heatmap = cv2.applyColorMap(heatmap,cv2.COLORMAP_JET)
        SaveImg(filename,filepath,heatmap)
    print ('********** Done ***********')

def SaveImg(filename,filepath,heatmap):
    img = cv2.imread(filepath)
    heatmap = cv2.resize(heatmap,(img.shape[1],img.shape[0]))
    AttentionImg =0.5* heatmap + img
    #in case file paths mess up
    if not cv2.imwrite(os.path.join('/content/drive/MyDrive/heatmap',filename),heatmap):
       raise Exception("Could not write heatmap")
    if not cv2.imwrite(os.path.join('/content/drive/MyDrive/AttentionImg',filename),AttentionImg):
       raise Exception("Could not write AttentionImg")

GAPAttention(model,weights, t_dir)
#ShowAttentionV1(base_model, t_dir)

# Crop the local patches for hand based on heatmaps


In [ ]:
#size
print(len(os.listdir(t_dir)))
print(len(os.listdir('/content/drive/MyDrive/heatmap')))
print(len(os.listdir('/content/drive/MyDrive/AttentionImg')))

In [ ]:
for i in os.listdir('/content/drive/MyDrive/Hand/'):
    #os.remove('/content/drive/MyDrive/Hand/'+i)

In [ ]:
print(len(list(os.listdir('/content/drive/MyDrive/Hand/'))))
print(len(list(os.listdir('/content/drive/MyDrive/heatmap/'))))

In [ ]:
def crop(img,mask):
    index = np.where(mask>0)
    top =np.min(index[0])
    bottom = np.max(index[0]) 
    left = np.min(index[1])
    right = np.max((index[1]))
   # extract hand region
   # if top > 200:
   #     top =top -200
   # elif top > 100:
   #     top = top -100

   # extract region1
   # if left>100:
   #     left=left-70

    croped_img = img[top:bottom,left:right]
    return croped_img



def maskout(img,mask):
    index = np.where(mask>0)
    top =np.min(index[0])
    bottom = np.max(index[0]) 
    left = np.min(index[1])
    right = np.max((index[1]))
    img[top:bottom,left:right]=np.random.randint(255)
    return img


def find_max_component(mask):
    contours, hierarchy = cv2.findContours(mask,cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)
    area = []
    for i in range(len(contours)):
        area.append(cv2.contourArea(contours[i]))
    try:
       max_ind = np.argmax(area)
    except ValueError: 
        print("Err")
        pass
    print (area)
    for ind in range(len(contours)):
        if ind != max_ind:
            cv2.fillConvexPoly(mask,contours[ind],0)
    return mask




if __name__=="__main__":
    errimg = []
    path_list = os.listdir('/content/drive/MyDrive/heatmap/')
    path_list.sort()
    kernel = np.ones((5,5),np.uint8)
    for path in path_list:
      if path.endswith('png'):
          img = cv2.imread(os.path.join(t_dir,path),0)
      try:
          heatmap = cv2.imread(os.path.join('/content/drive/MyDrive/heatmap/',path),0)
          #hand thres = 20
          ret,mask =  cv2.threshold(heatmap,20,255,cv2.THRESH_BINARY)
          mask = find_max_component(mask)
                # mask = cv2.dilate(mask,kernel,iterations=1)
                # cv2.imwrite('patches/'+path,mask)
                # img = img*mask
                # cv2.imwrite('patches/'+path,img)
          croped_img= crop(img,mask)
          if not cv2.imwrite(os.path.join('/content/drive/MyDrive/Hand/',path),croped_img):
                print("Exception: fail to write img " + path)
      except ValueError: 
          errimg.append(path)
          pass
          ##for deriving v2 
          #MaskImg = maskout(img,mask)
          #cv2.imwrite('Maskout/'+path,MaskImg)

In [ ]:
print(len(os.listdir('/content/drive/MyDrive/Hand')))

1319


In [ ]:
#deprecated maybe? not sure
print(errimg)

['3806.png', '6602.png']


# V1

In [ ]:
def SaveImg(filename,filepath,heatmap):
    img = cv2.imread(filepath)
    heatmap = cv2.resize(heatmap,(img.shape[1],img.shape[0]))
    AttentionImg =0.5* heatmap + img
    #in case file paths mess up
    if not cv2.imwrite(os.path.join('/content/drive/MyDrive/heatmap1',filename),heatmap):
       raise Exception("Could not write heatmap")
    if not cv2.imwrite(os.path.join('/content/drive/MyDrive/AttentionImg1',filename),AttentionImg):
       raise Exception("Could not write AttentionImg")


In [ ]:
base_model.summary()

In [ ]:
def ShowAttentionV1(model,image_path):
    file_list = os.listdir(image_path)
    file_list.sort()
    for filename in file_list:
        print (filename)
        filepath=image_path+filename
        image=load_image(filepath)
        image = image/255.0
        gender=1.0
        gender=np.asarray(gender)
        gender=np.expand_dims(gender,axis=0)
        #fed into a GMP layer which follow by the last FC layer.? but of the base model
        layer=K.function([model.layers[0].input],[model.layers[196].output])
        FeatureMap=layer([image])[0]
        print (FeatureMap.shape)
        FeatureMap = np.squeeze(FeatureMap, axis=0)
        FeatureMap = np.abs(FeatureMap)
        heatmap = np.mean(FeatureMap,axis=2)
        heatmap = heatmap/np.max(heatmap)
        heatmap = np.uint8(255*heatmap)
        print (heatmap.shape)
        heatmap = cv2.applyColorMap(heatmap,cv2.COLORMAP_JET)
        SaveImg(filename,filepath,heatmap)
    print ('********** Done ***********')

In [ ]:
print(model.summary)

<bound method Model.summary of <keras.engine.functional.Functional object at 0x7f8593b552d0>>


In [ ]:
#ShowAttentionV1(base_model, t_dir)

if __name__=="__main__":
    errimg1 = []
    path_list = os.listdir(t_dir)
    path_list.sort()
    print(path_list)
    kernel = np.ones((5,5),np.uint8)
    for path in path_list:
      if path.endswith('png'):
          img = cv2.imread(os.path.join(t_dir,path),0)
      try:
          heatmap = cv2.imread(os.path.join('/content/drive/MyDrive/heatmap/',path),0)
          #thres 50
          ret,mask =  cv2.threshold(heatmap,50,255,cv2.THRESH_BINARY)
          mask = find_max_component(mask)
                # mask = cv2.dilate(mask,kernel,iterations=1)
                # cv2.imwrite('patches/'+path,mask)
                # img = img*mask
                # cv2.imwrite('patches/'+path,img)
          print(path)
          croped_img= crop(img,mask)
          if not cv2.imwrite(os.path.join('/content/drive/MyDrive/V1/',path),croped_img):
                print("Exception: fail to write img" + path)
      except ValueError: 
          errimg1.append(path)
          pass

In [ ]:
print(len(list(os.listdir('/content/drive/MyDrive/AttentionImg1'))))
print(len(list(os.listdir('/content/drive/MyDrive/heatmap1'))))
print(len(list(os.listdir('/content/drive/MyDrive/V1/'))))

#ERASED

In [ ]:
def maskout(img,mask):
    index = np.where(mask>0)
    top =np.min(index[0])
    bottom = np.max(index[0]) 
    left = np.min(index[1])
    right = np.max((index[1]))
    img[top:bottom,left:right]=np.random.randint(255)
    return img


In [ ]:
#Create maskout images 
if __name__=="__main__":
    errimg2 = []
    path_list = os.listdir(t_dir)
    path_list.sort()
    #print(path_list)
    kernel = np.ones((5,5),np.uint8)
    for path in path_list:
      if path.endswith('png'):
          img = cv2.imread(os.path.join(t_dir,path),0)
      try:
          heatmap = cv2.imread(os.path.join('/content/drive/MyDrive/heatmap/',path),0)
          ret,mask =  cv2.threshold(heatmap,50,255,cv2.THRESH_BINARY)
          mask = find_max_component(mask)
          MaskImg = maskout(img,mask)
          cv2.imwrite(os.path.join('/content/drive/MyDrive/Maskout/',path), MaskImg)

                # mask = cv2.dilate(mask,kernel,iterations=1)
                # cv2.imwrite('patches/'+path,mask)
                # img = img*mask
                # cv2.imwrite('patches/'+path,img)
          #print(path)
          #croped_img= crop(img,mask)
          #if not cv2.imwrite(os.path.join('/content/drive/MyDrive/V1/',path),croped_img):
                #print("Exception: fail to write img" + path)
      except ValueError: 
          errimg2.append(path)
          pass
      

In [ ]:
#
print(len(list(os.listdir('/content/drive/MyDrive/Maskout/'))))

1319


#aggregate H+R1+E


In [ ]:
#save r1 e and h regions
f = open(root_dir+'data_age.pkl', 'rb')
age = pickle.load(f)
f.close()

f = open(root_dir+'data_gender.pkl','rb')
gender = pickle.load(f)
f.close()




In [ ]:
f = open(root_dir+'file.pkl', 'rb')
filelist = pickle.load(f)
f.close()

In [ ]:
#save r1 and e and hby this order
print(len(filelist))
filelist

In [ ]:
handpath = '/content/drive/MyDrive/Hand/'
v1path = '/content/drive/MyDrive/V1/'
maskout = '/content/drive/MyDrive/Maskout/'
mydrive = '/content/drive/MyDrive/'

In [ ]:
#hand
dataHand = []
for i in filelist:
    imgHand_path = handpath + i
    try:
      img = cv2.imread(imgHand_path)
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      img = cv2.resize(img,(560,560))
    except: 
        print("Err: "+i)
        pass
    else:
        x = np.asarray(img, dtype=np.uint8)
        dataHand.append(x)

dataHand_pkl = open(os.path.join(mydrive,'dataHand.pkl'),'wb')
cPickle.dump(dataHand, dataHand_pkl, protocol=cPickle.HIGHEST_PROTOCOL)
dataHand_pkl.close()
print ('100% completed saving Hand data')

In [ ]:
#R1
dataR1 = []
for i in filelist:
    try:
      imgR1_path = v1path + i
      img = cv2.imread(imgR1_path)
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      img = cv2.resize(img,(560,560))
    except: 
        print("Err: "+i)
        pass 
    else:
      if (i != '6602.png' and i != '3806.png'):
          x = np.asarray(img, dtype=np.uint8)
          dataR1.append(x)


In [ ]:
dataR1_pkl = open(os.path.join(mydrive,'dataR1.pkl'),'wb')
cPickle.dump(dataR1, dataR1_pkl, protocol=cPickle.HIGHEST_PROTOCOL)
dataR1_pkl.close()
print ('100% completed saving R1 data')


In [ ]:
#E
dataE = []
for i in filelist:
    imge_path = maskout + i
    try:
        img = cv2.imread(imge_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img,(560,560))
    except: 
        print("Err: "+i)
        pass 
    else:
      if (i != '6602.png' and i != '3806.png'):
          x = np.asarray(img, dtype=np.uint8)
          dataE.append(x)

dataE_pkl = open(os.path.join(mydrive,'dataE.pkl'),'wb')
cPickle.dump(dataE, dataE_pkl, protocol=cPickle.HIGHEST_PROTOCOL)
dataE_pkl.close()
print ('100% completed saving E data')

In [ ]:

f = open(os.path.join(root_dir,'data_age.pkl'), 'rb')
age = pickle.load(f)
f.close()

f = open(os.path.join(root_dir,'data_gender.pkl'),'rb')
gender = pickle.load(f)
f.close()

In [ ]:
#took out the two 6602 3806 update
print(list(filelist).index('6602.png'))
print(list(filelist).index('3806.png'))

In [ ]:
del age[961]
del age[892]


In [ ]:
del gender[961]
del gender[892]


#compile regression model

120epochs. learningrate set to 0.0003, 0.0001,and 0.00001 first60epochs,next30 epochs, final30epochs,respectively

In [ ]:

from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.layers import Flatten, Dense, Input, Reshape, Lambda
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
import pickle
import numpy as np
import matplotlib.pyplot as plt
import keras
import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"
os.environ['OMP_NUM_THREADS']='6'
batch_size = 16
epochs = 30

In [ ]:
# Load data no need to run
mydrive = '/content/drive/MyDrive/'
print('...loading training data')
f = open(os.path.join(mydrive,'dataE.pkl'), 'rb')
dataE = pickle.load(f)
f.close()

f = open(os.path.join(mydrive,'dataR1.pkl'), 'rb')
dataR1 = pickle.load(f)
f.close()

f = open(os.path.join(mydrive,'dataHand.pkl'), 'rb')
dataHand = pickle.load(f)
f.close()


...loading training data


In [ ]:
 #  Using VGG19 with pretrained weights from Imagenet 
base_model = Xception(weights='imagenet', include_top=False)
for i,layer in enumerate(base_model.layers):
    print (i,layer.name)
input = Input(shape=(560,560,3),name='input1')
input_gender = Input(shape=(1,),dtype='float32',name='input2')
output = base_model(input)
gender_embedding=Dense(32)(input_gender)
#gender_embedding=Dense(12)(gender_embedding)
#x = keras.layers.MaxPooling2D(pool_size=(5,5))(output)
#x = keras.layers.Conv2D(512,kernel_size=(3,3))(x)
x = keras.layers.Conv2D(256,kernel_size=(3,3))(output)
print (K.int_shape(output))
x = keras.layers.MaxPooling2D(pool_size=(3,3))(x)
print (K.int_shape(x))
x=Flatten()(x)
f = keras.layers.Concatenate(axis=1)([x,gender_embedding])
print (K.int_shape(f)) 
#x = Dense(256, activation='relu')(x)
predictions = Dense(1)(f)




In [ ]:
model = Model(inputs=[input,input_gender], outputs=predictions)
for i,layer in enumerate(model.layers):
    print (i,layer.name)

Adam=tensorflow.keras.optimizers.Adam(lr=0.0003,beta_1=0.9,beta_2=0.999)
model.compile(optimizer=Adam, loss='mean_absolute_error', metrics=['MAE'])

In [ ]:
# Save weights after every epoch
DataGen = ImageDataGenerator(rotation_range=20,width_shift_range=0.15,height_shift_range=0.15,zoom_range=0.2,horizontal_flip=True)
def Generator(x_train,gender_train,y_train,batch_size):
    loopcount = len(y_train)//batch_size
    i=0
    while (True):
        if i>loopcount:
            i=0
        # i=np.random.randint(0,loopcount)
        x_train_batch = x_train[i*batch_size:(i+1)*batch_size,:,:,:]
        x_train_batch = DataAugment(x_train_batch)
        gender_train_batch = gender_train[i*batch_size:(i+1)*batch_size]
        y_train_batch = y_train[i*batch_size:(i+1)*batch_size]
        inputs = [x_train_batch,gender_train_batch]
        target = y_train_batch
        yield (inputs ,target)
        i = i+1

In [ ]:
data = np.asarray(dataHand, dtype=np.float32)
print (data.shape)

dataR1 = np.asarray(dataR1, dtype=np.float32)
dataE = np.asarray(dataE, dtype=np.float32)
#data[:,:,:,0] = dataR1[:,:,:,0]
data[:,:,:,1] = dataR1[:,:,:,1]
data[:,:,:,2] = dataE[:,:,:,2]
print (dataR1.shape)

age = np.asarray(age)
gender = np.asarray(gender)
print (age.shape)

In [ ]:

data /= 255.
gender =2*( gender-0.5)
x_final = []
y_final = []
gender_final = []

# Shuffle images and split into train, validation and test sets
#random_no = np.random.choice(data.shape[0], size=data.shape[0], replace=False)
random_no = np.arange(dataR1.shape[0])
np.random.seed(0)
np.random.shuffle(random_no)
for i in random_no:
    x_final.append(data[i,:,:,:])
    y_final.append(age[i])
    gender_final.append(gender[i])

x_final = np.asarray(x_final)
y_final = np.asarray(y_final)
gender_final = np.asarray(gender_final)
#print (y_final[:50])
#print (gender_final[:50])
k = 55 # Decides split count
x_test = x_final[:k,:,:,:]
y_test = y_final[:k]
gender_test = gender_final[:k]
x_valid = x_final[k:2*k,:,:,:]
y_valid = y_final[k:2*k]
gender_valid = gender_final[k:2*k]
x_train = x_final[2*k:,:,:,:]
y_train = y_final[2*k:]
gender_train = gender_final[2*k:]


In [ ]:
del data
del dataR1
del dataE
del x_final

## 
#y_test = keras.utils.to_categorical(y_test,240)
#y_train = keras.utils.to_categorical(y_train,240)
#y_valid = keras.utils.to_categorical(y_valid,240)
#y_train = softlabel(y_train,240)
#y_valid = softlabel(y_valid,240)
#y_test = softlabel(y_test,240)


print ('x_train shape:'+ str(x_train.shape))
print ('y_train shape:'+ str(y_train.shape))
print ('gender_train shape:'+ str(gender_train.shape))
print ('x_valid shape:'+ str(x_valid.shape))
print ('y_valid shape:'+ str(y_valid.shape))
print ('gender_valid shape:' + str(gender_valid.shape))
print ('x_test shape:'+ str(x_test.shape))
print ('y_test shape:'+ str(y_test.shape))

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint(filepath=os.path.join(mydrive,'weights2/weights.{epoch:02d}-{val_loss:.2f}.hdf5'),save_weights_only=True)
history = model.fit_generator(DataGen.flow([x_train,gender_train],y_train,batch_size=batch_size),steps_per_epoch=np.ceil(len(y_train)/batch_size),epochs=40,verbose=1,validation_data=([x_valid,gender_valid],y_valid))
#history = model.fit_generator(Generator(x_train,gender_train,y_train,batch_size),steps_per_epoch=np.ceil(len(y_train)/batch_size),epochs=10,verbose=1,validation_data=([x_valid,gender_valid],y_valid))
#history = model.fit([x_train,gender_train],y_train,batch_size=batch_size,epochs=80,verbose=1,validation_data=([x_valid,gender_valid],y_valid), callbacks = [checkpoint])
score = model.evaluate([x_test,gender_test], y_test, batch_size=batch_size)
print('Test loss:', score[0])
print('Test MAE:', score[1])

##Visulization
weights=model.layers[-1].get_weights()[0]
print (weights.shape)

In [ ]:
#past weights
os.listdir('/content/drive/MyDrive/weight2/')

In [ ]:
TestMAE = TestMAE(model,x_test,y_test,gender_test)
print ('TestMAE:',TestMAE)

In [ ]:
#part2
#for layer in base_model.layers[:16]:
#    layer.trainable=False
#for layer in base_model.layers:
#    print (layer.name,layer.trainable)
Adam=tensorflow.keras.optimizers.Adam(lr=0.0001,beta_1=0.9,beta_2=0.999)
model.compile(optimizer=Adam, loss='mean_absolute_error', metrics=['MAE'])
#history = model.fit_generator(Generator(x_train,gender_train,y_train,batch_size),steps_per_epoch=np.ceil(len(y_train)/batch_size),epochs=30,verbose=1,validation_data=([x_valid,gender_valid],y_valid))
history = model.fit([x_train,gender_train], y_train,batch_size=batch_size,epochs=20,verbose=1,validation_data=([x_valid,gender_valid],y_valid), callbacks = [checkpoint])
score = model.evaluate([x_test,gender_test], y_test, batch_size=batch_size)
print('Test loss:', score[0])
print('Test MAE:', score[1])

In [ ]:
model.save_weights(os.path.join(mydrive,"modelagg.h5")
with open('historyagg.pkl', 'wb') as f:
	pickle.dump(history.history, f)
f.close()